Example of text analysis for Taiwan National Treasure (TNT) data
1. read data from api
2. structure ocr and nlp data frame

In [1]:
import urllib2 as ul
import requests, json
import pandas as pd
NLAPI = str("https://api.nationaltreasure.tw/v1/upload")

Request single data from API

In [121]:
data = ul.urlopen(NLAPI)

parsed_data = json.loads(data.read())
count = parsed_data['Count']
Items = parsed_data['Items']

uid_list = [u.get('uid') for u in Items]

link = NLAPI+'?uid='+uid_list[0]
for uid in uid_list[:5]:
    print NLAPI+'?uid='+uid

doc = json.load(ul.urlopen(link))
pd.DataFrame(doc)

https://api.nationaltreasure.tw/v1/upload?uid=1f5cd880-88eb-11e7-b276-4d7eb1b5b1eb
https://api.nationaltreasure.tw/v1/upload?uid=347849c0-88eb-11e7-b276-4d7eb1b5b1eb
https://api.nationaltreasure.tw/v1/upload?uid=39ba1d00-88eb-11e7-8cef-31d850aa17ef
https://api.nationaltreasure.tw/v1/upload?uid=3cb29c80-88eb-11e7-8cef-31d850aa17ef
https://api.nationaltreasure.tw/v1/upload?uid=3cd9ac80-88eb-11e7-b3cb-0fa397c90bdd


,Item
docId,3303795
endYear,1980
imageKey,NARA/3303795/1f5cd880-88eb-11e7-b276-4d7eb1b5b...
isNotDocument,False
isReadyForView,False
location,NARA
metadata,"{u'box': u'62-63', u'containerId': u'62-63', u..."
nlpEn,"[{u'entities': [{u'name': u'COMMEPCIAL', u'sen..."
nlpZh,[{u'entities': [{u'name': u'\u5168\u9ad4\u6703...
ocr,[DECLASSIFIED\nFLE\nTO WHITE HOUSE\nSECSTATE\n...


Read first 1000 data

In [67]:
docs = {}
index = 0 
for uid in uid_list:
    link = NLAPI+'?uid='+uid
    temp = json.load(ul.urlopen(link))
    docs[index]=temp['Item']
    index+=1

columns = doc.keys
df = pd.DataFrame.from_dict(docs,orient='index')

df.info()

In [94]:
ocr = df[['ocr']] #save for later

In [86]:
df.loc[0].nlpEn[0]['entities'][0].keys()

[u'name', u'sentiment', u'salience', u'mentions', u'type', u'metadata']

In [179]:
cols = ['uid','name','type','salience','count']
nlp = []
for i,d in enumerate(df['nlpEn']):
    uid = df['uid'][i]
    if len(d)!=0:
        for p in d[0]['entities']:
            count = len(p['mentions'])
            name = p['name'].lower()
            t = p['type']
            s = p['salience']
            nlp.append(dict( zip( cols, [uid,name,t,s,count])))
ds_nlp = pd.DataFrame(nlp)    

In [180]:
ds_nlp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48726 entries, 0 to 48725
Data columns (total 5 columns):
count       48726 non-null int64
name        48726 non-null object
salience    48726 non-null float64
type        48726 non-null object
uid         48726 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 1.9+ MB


There are some missing data

In [181]:
ds_nlp.uid.nunique()

970

Remove duplicate terms and common words

In [185]:
ds_nlp.name.value_counts()[:10]

relations                 876
people                    795
president                 645
taiwan                    540
declassified authority    462
section                   392
united states             357
government                353
bill                      254
law                       251
Name: name, dtype: int64

top 5 name in each type catalog

In [193]:
ds_nlp.groupby('type').count()

,count,name,salience,uid
type,,,,
CONSUMER_GOOD,276,276,276,276
EVENT,1601,1601,1601,1601
LOCATION,4336,4336,4336,4336
ORGANIZATION,6328,6328,6328,6328
OTHER,27358,27358,27358,27358
PERSON,7044,7044,7044,7044
UNKNOWN,5,5,5,5
WORK_OF_ART,1778,1778,1778,1778


In [207]:
ty = ['EVENT','LOCATION']
for t,group in ds_nlp.groupby('type'):
    print t
    print group.name.value_counts()[:5]

CONSUMER_GOOD
vehicle    16
cars        9
car         9
items       8
weapons     7
Name: name, dtype: int64
EVENT
normalization    101
attack            72
retirement        52
talks             51
negotiations      46
Name: name, dtype: int64
LOCATION
taiwan           540
united states    289
u.s.             168
state            152
country          134
Name: name, dtype: int64
ORGANIZATION
declassified authority    462
government                353
congress                  194
agency                    171
administration            137
Name: name, dtype: int64
OTHER
relations    876
law          251
basis        199
programs     195
security     186
Name: name, dtype: int64
PERSON
people         795
president      645
employees      122
authorities    110
employee        98
Name: name, dtype: int64
UNKNOWN
taipei                                     1
embassy of the united states of america    1
台北市                                        1
美國                                        